In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense, SimpleRNN, LSTM, GRU, Bidirectional
from tensorflow.keras.utils import to_categorical

iris = load_iris()
X = iris.data
y = iris.target

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

ml_results = {}

models = {
    "Logistic Regression": LogisticRegression(max_iter=1000),
    "SVM": SVC(),
    "Random Forest": RandomForestClassifier(n_estimators=100, random_state=42),
    "XGBoost": XGBClassifier(eval_metric="mlogloss", random_state=42)  # ✅ removed use_label_encoder
}

for name, model in models.items():
    model.fit(X_train, y_train)
    preds = model.predict(X_test)
    acc = accuracy_score(y_test, preds)
    ml_results[name] = acc

dl_results = {}

# MLP (Sklearn)
mlp = MLPClassifier(hidden_layer_sizes=(64, 32), max_iter=2000, random_state=42)
mlp.fit(X_train, y_train)
preds = mlp.predict(X_test)
dl_results["MLP (Sklearn)"] = accuracy_score(y_test, preds)

# TensorFlow Models (need categorical labels)
y_train_cat = to_categorical(y_train, num_classes=3)
y_test_cat = to_categorical(y_test, num_classes=3)

X_train_rnn = np.expand_dims(X_train, axis=1)
X_test_rnn = np.expand_dims(X_test, axis=1)


def build_and_train(model, name):
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    model.fit(X_train_rnn, y_train_cat, epochs=50, verbose=0)
    _, acc = model.evaluate(X_test_rnn, y_test_cat, verbose=0)
    dl_results[name] = acc


# RNN
rnn = Sequential([
    Input(shape=(1, 4)),
    SimpleRNN(32),
    Dense(3, activation='softmax')
])
build_and_train(rnn, "RNN")

# LSTM
lstm = Sequential([
    Input(shape=(1, 4)),
    LSTM(32),
    Dense(3, activation='softmax')
])
build_and_train(lstm, "LSTM")

# BiLSTM
bilstm = Sequential([
    Input(shape=(1, 4)),
    Bidirectional(LSTM(32)),
    Dense(3, activation='softmax')
])
build_and_train(bilstm, "BiLSTM")

# GRU
gru = Sequential([
    Input(shape=(1, 4)),
    GRU(32),
    Dense(3, activation='softmax')
])
build_and_train(gru, "GRU")

# BiGRU
bigru = Sequential([
    Input(shape=(1, 4)),
    Bidirectional(GRU(32)),
    Dense(3, activation='softmax')
])
build_and_train(bigru, "BiGRU")

print("\n--- Machine Learning Models Accuracy ---")
print(pd.Series(ml_results))

print("\n--- Deep Learning Models Accuracy ---")
print(pd.Series(dl_results))


--- Machine Learning Models Accuracy ---
Logistic Regression    0.966667
SVM                    0.966667
Random Forest          0.900000
XGBoost                0.933333
dtype: float64

--- Deep Learning Models Accuracy ---
MLP (Sklearn)    1.000000
RNN              0.966667
LSTM             0.966667
BiLSTM           1.000000
GRU              0.966667
BiGRU            1.000000
dtype: float64
